** GOAL **

To reproduce the number from the Austen report.  Numbers will be approximate because we will use averages for tuition paid, admin cost, total credits taken per student, etc.

** DEFINITIONS **

* total student credit hours = (number of students) x (credits for course)


** COST **

* cost = (contact hours)/12 x salary # use ave salary by rank

* admin cost = (department head salary/4 + ? (1/2 of george's salary, maybe admin assist, dean)) / (total number of PHYS courses)

* total costs = cost + admin cost + (department budget / total number of PHYS courses) 

* cost per SCH = (total cost) / total student credit hours


** REVENUE **

* revenue = (tuition x (1-discount rate))/(total credits taken) x (credits for course) # use average

* revenue per SCH = revenue / total student credit hours


** MARGIN **

* margin = revenue - cost

* margin per SCH = (revenue per SCH) - (cost per SCH)

* department margin = sum(revenue) - sum(cost)



** PREPARING THE DATA ** 

* we added two columns to the spreadsheet that Angela sent
    * credit hours (column 15)
    * contact hrs (column 16)
    




In [232]:
import xlrd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [233]:
def readSchedule(file,match_string):
    block=[]
    days=[]
    times=[]
    room=[]
    fac=[]
    credit=[]
    course=[]
    mathcourse=[]
    mathtitle=[]
    mathblock=[]
    mathdays=[]
    mathtimes=[]
    credit_hrs = []
    contact_hrs = []
    enrollment = []
    if file.find('.xls') > -1:# reading an excel file
        xdat=xlrd.open_workbook(file)
        sheet=xdat.sheet_by_index(0)
        ncol=sheet.ncols
        nrow=sheet.nrows
        # find course index
        for i in range(nrow):
            coursename=sheet.cell_value(i,5)
            if coursename.startswith(match_string):# | coursename.startswith('ASTR') : #| (coursename.find('SCDV001') > -1)  | (coursename.find('CSIS200') > -1) | (coursename.find('SCDV230') > -1):
                enrollment.append(sheet.cell_value(i,3))
                course.append(sheet.cell_value(i,5))
                block.append(sheet.cell_value(i,7))
                days.append(sheet.cell_value(i,8))
                times.append(sheet.cell_value(i,9))
                room.append(sheet.cell_value(i,10))
                fac.append(sheet.cell_value(i,11).split(',')[0])
                credit.append(sheet.cell_value(i,12))
                credit_hrs.append(sheet.cell_value(i,14))
                contact_hrs.append(sheet.cell_value(i,15))
        #xdat=xlrd.close_workbook(file)
        return block,room,fac,credit,course,days,times,credit_hrs,contact_hrs,enrollment
    else:
        for line in file:
            if line.startswith('CRN'):
                continue
            t=line.split(',')
            try:
                course.append(t[2])
                block.append(t[4])
                room.append(t[7])
                fac.append(t[8])
                credit.append(t[9])
            except:
                print "Error reading line: ",line
                continue
        return block,room,fac,credit,course,days,times,credit_hrs,contact_hrs 

In [292]:
class department:
    def __init__(self,prefix,infile):
        self.prefix = prefix
        if prefix.find('PHYS') > -1: # read file twice to get PHYS and ASTR courses
            block,room,fac,credit,course,days,times,credit_hrs,contact_hrs,enrollment = readSchedule(infile,prefix)
            t = readSchedule(infile,'ASTR')
            self.faculty = np.array((fac + t[2]),'S16')
            self.course = np.array((course + t[4]),'S7')
            self.credit_hrs = np.array((credit_hrs + t[7]),'f')
            self.contact_hrs = np.array((contact_hrs + t[8]),'f')
            self.enrollment = np.array((enrollment + t[9]),'f')
        else:
            block,room,fac,credit,course,days,times,credit_hrs,contact_hrs,enrollment = readSchedule(infile,prefix)
            self.faculty = np.array((fac),'S16')
            self.course = np.array((course),'S7')
            self.credit_hrs = np.array((credit_hrs),'f')
            self.contact_hrs = np.array((contact_hrs),'f')
            self.enrollment = np.array((enrollment),'f')
        self.calc_cost()
        self.calc_revenue()
        self.calc_margin()
    def calc_cost(self):
        self.cost_per_course = np.zeros(len(self.faculty))
        nhassel = 0
        for i in range(len(self.faculty)):
            #print faculty[i]
            #print salary_dict[self.faculty[i]]
            try:
                if self.faculty[i].find('Hassel') > -1:
                    nhassel += 1
                    if nhassel < 2:
                        self.cost_per_course[i] = salary_dict[self.faculty[i]]/24.*self.contact_hrs[i]*1.4
                    else:
                        self.cost_per_course[i] = self.contact_hrs[i] * salary_adjunct
                else:
                    self.cost_per_course[i] = salary_dict[self.faculty[i]]/24.*self.contact_hrs[i]*1.4
        
            except KeyError: # enter here for adjunct faculty
                #print 'Key error for ',i,self.faculty[i]
                #if faculty[i].find('Gig') > -1:
                #    print 'Key error for ', self.faculty[i]
                #else:
                #    print salary_dict[self.faculty[i]]
                #print 'got here!'
                print 'assuming ',self.faculty[i],' is an adjunct professor who teaches ',np.sum(self.contact_hrs[self.faculty == self.faculty[i]]),' contact hrs'
                self.cost_per_course[i] = self.contact_hrs[i]* salary_adjunct# course credit hrs * salary per credit hour
        self.total_cost = np.sum(self.cost_per_course) + department_budget[self.prefix]
        self.cost_per_sch = self.total_cost/np.sum(self.credit_hrs*self.enrollment)
    def calc_revenue(self):
        self.revenue_per_course = tuition_paid/30.*self.credit_hrs*self.enrollment
        self.total_revenue = np.sum(self.revenue_per_course)
    def calc_margin(self):
        self.margin_per_course = self.revenue_per_course - self.cost_per_course
        self.total_margin = self.total_revenue - self.total_cost
    def print_stats(self):
        print '\n%%%%%%%%%%%%%%%%%%%%%%%'
        print self.prefix
        print '%%%%%%%%%%%%%%%%%%%%%%%'
        print 'average cost per class    = %5.2f'%(np.mean(self.cost_per_course))
        print 'average revenue per class = %5.2f'%(np.mean(self.revenue_per_course))
        print 'average margin per class  = %5.2f'%(np.mean(self.margin_per_course))
        print 'average class size        = %3.1f'%(np.mean(self.enrollment))
    def print_stats_alt(self):

        print '%s %5.2f %5.2f %5.2f %3.1f'%(self.prefix,np.mean(self.cost_per_course), 
                                            np.mean(self.revenue_per_course), np.mean(self.margin_per_course),
                                            np.mean(self.enrollment))
        
        

In [293]:
# assumptions

# faculty salaries
salary_assistant = 70000.
salary_associate = 80000.
salary_full = 95000.
salary_adjunct = 850. #per credit hour
salary_visitor = 65000.

faculty_ranks = {'Finn':salary_associate,'Coohill':salary_full,'Vernizzi':salary_associate,
                'McColgan':salary_assistant,'Cummings':salary_associate,'Rosenberry':salary_associate,
                'Hassel':salary_assistant,'Moustakas':salary_assistant,'Weatherwax':salary_full,
                 'Medsker':salary_full,'Bellis':salary_assistant,
                 'Caldaro':salary_visitor,'Russell':salary_visitor}
assistants = ['Bellis','McColgan','Moustakas','Hassel', # Physics
              #Biology
              'Vernooy',
              'Swinton',
              'Goldman',
              'Brookins', 
              'Berke',
              'Springer',
              'Karr','Deyrup',# Computer Science
              'Fryling','DiTursi','Small',# Computer Science
              'Meierdiercks','Kolozsvary', #Env
              'Javaheri', 'Henry' # Math
              ]
              
associates = ['Finn','Cummings','Vernizzi','Rosenberry', # Physics
              'Memmo-Signor','Mason','Harbison','Byrnes', #Biology
              # Chemistry
              'Hofstein','Tucker','Moriarty','Barnes',"O'Donnell",'Hughes','Kolonko',
              'Lim','Cotler','Berman','Breimer',# Computer Science
              'Ellard', #ENVA
              'Bannon','Krylov',"O'Neill" #Math
              ]
full = ['Coohill','Medsker','Weatherwax', #Physics
        'Woolbright','Angstadt','Helm','Hayden','Wilson','Worthington','Zanetti','Sterne-Marr','LaRow', #Biology
        'Rhoads', # Chemistry
        'Egan','Vandenberg','Horowitz','Flatland','Matthews', # Computer Science
        'Mangun', #Env
        'Kenney', 'Rogers' #Math
       ]
visitors = ['Caldaro','Russell',
           'Rapp', 'Pier','Chaturvedi', #Biology
            'Wos','Barbera','LaGraff','McNamara','Lee','Vanderover','Perry', #Chemistry
            'Liss','Sherman','Mehta','Goldstein','Yates', #Computer Science
            'Bogan', #Environmental Science
            'Smitas', 'Cade', #Math
            ]
adjuncts = ['Broder','Gigante', #Physics 
           #Biology
          "O'Brien",#Chemistry
            'Coco','Mendez','Rivituso','Schindler','Baciewicz', #Computer Science
            'Pipino', #ENVA
            'Kiehle','Mazzone' #Math
           ]

# make a dictionary of faculty and rank
nfaculty = len(assistants) + len(associates) + len(full) + len(visitors)
faculty_salaries = np.zeros(nfaculty,'f')
faculty_salaries[0:len(assistants)] = salary_assistant*np.ones(len(assistants))
faculty_salaries[len(assistants):len(assistants)+len(associates)] = salary_associate*np.ones(len(associates))
faculty_salaries[len(assistants)+len(associates):len(assistants)+len(associates)+len(full)] = salary_full*np.ones(len(full))
faculty_salaries[len(assistants)+len(associates)+len(full):] = salary_visitor*np.ones(len(visitors))
# self.nsadict=dict((a,b) for a,b in zip(self.s.NSAID,arange(len(self.s.NSAID))))
salary_dict = dict((a,b) for a,b in zip(assistants+associates+full+visitors,faculty_salaries))
# student numbers
discount_rate = .65
tuition = 32000.
tuition_paid = tuition * (1.-discount_rate)

# physics budget
physics_budget = 35000.

department_budget = {'PHYS':(35000.+35000.),'BIOL':150000.,'CSIS':150000.,'MATH':10000.,'CHEM':80000.,'ENVA':20000.}

# tuition
tuition = 32000.

#discount rate

discount_rate = 0.6

tuition_paid = tuition*(1-discount_rate)


In [294]:
infile = 'Fall2014.xls'
phys = department('PHYS',infile)

assuming  Gigante  is an adjunct professor who teaches  3.0  contact hrs
assuming  Broder  is an adjunct professor who teaches  6.0  contact hrs
assuming  Serbalik  is an adjunct professor who teaches  4.0  contact hrs
assuming  Serbalik  is an adjunct professor who teaches  4.0  contact hrs
assuming  Serbalik  is an adjunct professor who teaches  4.0  contact hrs
assuming  Broder  is an adjunct professor who teaches  6.0  contact hrs


In [295]:
bio = department('BIOL',infile)
chem = department('CHEM',infile)
math = department('MATH',infile)
cs = department('CSIS',infile)
enva = department('ENVA',infile)

assuming  O'Brien  is an adjunct professor who teaches  3.0  contact hrs
assuming  Serbalik  is an adjunct professor who teaches  3.0  contact hrs
assuming  Kiehle  is an adjunct professor who teaches  6.0  contact hrs
assuming  Kiehle  is an adjunct professor who teaches  6.0  contact hrs
assuming  Mazzone  is an adjunct professor who teaches  9.0  contact hrs
assuming  Mazzone  is an adjunct professor who teaches  9.0  contact hrs
assuming  Mazzone  is an adjunct professor who teaches  9.0  contact hrs
assuming    is an adjunct professor who teaches  21.0  contact hrs
assuming  Coco  is an adjunct professor who teaches  3.0  contact hrs
assuming    is an adjunct professor who teaches  21.0  contact hrs
assuming  Mendez  is an adjunct professor who teaches  3.0  contact hrs
assuming    is an adjunct professor who teaches  21.0  contact hrs
assuming  Rivituso  is an adjunct professor who teaches  3.0  contact hrs
assuming  Schindler  is an adjunct professor who teaches  9.0  contact hr

In [297]:
print phys.cost_per_sch
print phys.total_cost

284.700407213
489400.0


In [271]:
depts = [phys, bio, chem, math, cs, enva]

In [266]:
for d in depts:
    print d.print_stats_alt()

PHYS 6639.13 15944.35 9305.22 15.6
None
BIOL 10154.22 13897.14 3742.92 15.6
None
CHEM 9462.64 12437.33 2974.69 16.7
None
MATH 8960.20 15447.07 6486.87 16.6
None
CSIS 8509.94 11031.65 2521.71 16.5
None
ENVA 9198.81 10422.86 1224.05 11.0
None


In [57]:
# revenue
student_cost_per_class = tuition_paid/30.*3
nstudents_needed = np.mean(cost_per_course)/student_cost_per_class
print nstudents_needed, np.mean(enrollment[enrollment > 5.])

11.4596273292 21.0606060606


In [50]:
for i in range(len(course)):
    print course[i],cost_per_course[i], cost_per_course[i]/(enrollment[i]*credit_hrs[i])

ASTR010 17500.0 171.568627451
ASTR010 2550.0 22.3684210526
ASTR010 16250.0 159.31372549
ASTR010 20000.0 190.476190476
PHYS010 20000.0 196.078431373
PHYS010 16250.0 246.212121212
PHYS010 16250.0 159.31372549
PHYS110 17500.0 175.0
PHYS110 2550.0 inf
PHYS110 17500.0 182.291666667
PHYS110 2550.0 inf
PHYS110 17500.0 198.863636364
PHYS110 2550.0 inf
PHYS110 0.0 nan
PHYS120 16250.0 290.178571429
PHYS120 2550.0 inf
PHYS130 20000.0 inf
PHYS130 20000.0 238.095238095
PHYS130 20000.0 inf
PHYS130 20000.0 277.777777778
PHYS130 20000.0 inf
PHYS130 17500.0 198.863636364
PHYS132 6666.66666667 inf
PHYS132 6666.66666667 inf
PHYS220 2550.0 inf
PHYS220 20000.0 208.333333333
PHYS220 20000.0 inf
PHYS222 20000.0 2500.0
PHYS310 23333.3333333 388.888888889
PHYS310 23333.3333333 388.888888889
PHYS310 23333.3333333 388.888888889
PHYS310 23333.3333333 388.888888889
PHYS390 23333.3333333 416.666666667
PHYS400 0.0 nan
PHYS440 26666.6666667 1333.33333333
PHYS470 20000.0 2857.14285714
PHYS499 5833.33333333 5833.333333

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


In [70]:
help(xlrd)

Help on package xlrd:

NAME
    xlrd

FILE
    //anaconda/lib/python2.7/site-packages/xlrd/__init__.py

PACKAGE CONTENTS
    biffh
    book
    compdoc
    formatting
    formula
    info
    licences
    sheet
    timemachine
    xldate
    xlsx

FUNCTIONS
    count_records(filename, outfile=<ipykernel.iostream.OutStream object>)
    
    dump(filename, outfile=<ipykernel.iostream.OutStream object>, unnumbered=False)
    
    open_workbook(filename=None, logfile=<ipykernel.iostream.OutStream object>, verbosity=0, use_mmap=1, file_contents=None, encoding_override=None, formatting_info=False, on_demand=False, ragged_rows=False)

DATA
    FMLA_TYPE_ARRAY = 4
    FMLA_TYPE_CELL = 1
    FMLA_TYPE_COND_FMT = 8
    FMLA_TYPE_DATA_VAL = 16
    FMLA_TYPE_NAME = 32
    FMLA_TYPE_SHARED = 2
    MMAP_AVAILABLE = 1
    USE_MMAP = 1
    XL_CELL_BLANK = 6
    XL_CELL_BOOLEAN = 4
    XL_CELL_DATE = 3
    XL_CELL_EMPTY = 0
    XL_CELL_ERROR = 5
    XL_CELL_NUMBER = 2
    XL_CELL_TEXT = 1
    __VERSION